# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(131)
initial_params = np.random.random([13])

INITIALIZATION_METHOD = 'Amplitude'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label in [0, 1, 2, 3]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n-1] for n in validation_indexes]
y_validation = [y_train[n-1] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=3)

In [9]:
def unitary(params, wire1, wire2):
    # qml.RZ(0, wires=wire1)
    qml.RY(params[0], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[1], wires=wire2)
    # qml.RZ(0, wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[2], wires=wire2)
    qml.CNOT(wires=[wire1, wire2])
    qml.RY(params[3], wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[4], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[5], wires=wire2)
    # qml.RZ(0, wires=wire2)

In [10]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(3), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(3), rotation='Y')

    # First layer
    unitary(params[0:6], 0, 1)

    # Second layer
    unitary(params[6:12], 1, 2)

    # Third layer
    qml.RY(params[12], wires=2)

    # Measurement
    return qml.expval(qml.PauliZ(2))

## Circuit example

In [11]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205 0.12660931
 0.23946678 0.25415707 0.42644165 0.83900255 0.74503365 0.38067928
 0.26169292]

Example features: [-2.13370975 -1.89235125  2.16298094  0.8329378   1.03914201  2.89509077
 -2.82213403  0.1741292 ]

Expectation value: 0.5596790711246593

 0: ──╭QubitStateVector(M0)──RY(0.65)───╭X─────────────╭C─────────────╭X──RY(0.691)─────────────────────────────────────────────────────────────────────┤     
 1: ──├QubitStateVector(M0)──RY(0.948)──╰C──RY(0.388)──╰X──RY(0.641)──╰C──RY(0.127)──RY(0.239)──╭X─────────────╭C─────────────╭X──RY(0.745)─────────────┤     
 2: ──╰QubitStateVector(M0)──RY(0.254)──────────────────────────────────────────────────────────╰C──RY(0.426)──╰X──RY(0.839)──╰C──RY(0.381)──RY(0.262)──┤ ⟨Z⟩ 
M0 =
[-0.38346   +0.j -0.34008421+0.j  0.38872047+0.j  0.14969155+0.j
  0.18674958+0.j  0.52029171+0.j -0.50718028+0.j  0.03129366+0.j]



# Accuracy test definition

In [12]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [13]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.5387755102040817
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.5968787515006002
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.6218487394957983
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.665546218487395
best accuracy so far!
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.6609843937575031
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.680672268907563
best accuracy so far!
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.6861944777911164
best accuracy so far!
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.7123649459783914
best accu

epoch 341
epoch 342
epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.719327731092437
test_accuracies: [0.6861944777911164, 0.7123649459783914, 0.7214885954381753, 0.7262905162064826, 0.7289315726290516, 0.7294117647058824, 0.7226890756302521, 0.7306122448979592, 0.7226890756302521, 0.7279711884753901, 0.7234093637454981, 0.7181272509003602, 0.726530612244898, 0.7327731092436975, 0.7258103241296519, 0.7260504201680672, 0.7133253301320528, 0.729171668667467, 0.7373349339735895, 0.7298919567827131, 0.7332533013205282, 0.7255702280912365, 0.7320528211284514, 0.7162064825930372, 0.704921968787515, 0.7214885954381753, 0.7135654261704683, 0.7104441776710684, 0.731812725090036, 0.719327731092437]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.7260504201680672
test_accuracies: [0.7123649459783914, 0.7214885954381753, 0.7262905162064826, 0.7289315726290516, 0.7294117647058824, 0.7226890756

epoch 455
epoch 456
epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.7289315726290516
test_accuracies: [0.7181272509003602, 0.726530612244898, 0.7327731092436975, 0.7258103241296519, 0.7260504201680672, 0.7133253301320528, 0.729171668667467, 0.7373349339735895, 0.7298919567827131, 0.7332533013205282, 0.7255702280912365, 0.7320528211284514, 0.7162064825930372, 0.704921968787515, 0.7214885954381753, 0.7135654261704683, 0.7104441776710684, 0.731812725090036, 0.719327731092437, 0.7260504201680672, 0.7298919567827131, 0.7224489795918367, 0.7270108043217287, 0.7205282112845138, 0.7272509003601441, 0.7301320528211285, 0.7222088835534214, 0.7332533013205282, 0.7358943577430972, 0.7289315726290516]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.7296518607442977
test_accuracies: [0.726530612244898, 0.7327731092436975, 0.7258103241296519, 0.7260504201680672, 0.7133253301320528, 0.729171668667467, 0.7373349339735895, 0.72989195678

In [14]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 0.26563633  1.66502588  1.62440585  0.10308565  0.69051205  0.19301864
  0.30587611 -0.71334516  2.2251851   0.40209308  0.74503365  0.02780758
 -0.09117878]


# Testing

In [15]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.7453333333333334


In [16]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 14.912296533584595
training time: 1695.5147542953491
test time: 62.28123927116394
total time: 1772.7082901000977
